In [2]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import talib
from plotly.subplots import make_subplots
import pickle
from tqdm.notebook import tqdm
import copy
import warnings

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

from functions_TA_indicator import *

### Transformations

We will firstly transform 'bollinger_bands', 'ema', 'macd', 'max_high', 'min_low', 'roc', 'rsi', by letting them be the percentage difference of the close price.

Next we will subtract 50 from all of the occilators: 'aroon', 'rsi'.

We wont change 'macd' as it already occilates arround 0, but has no upper/lower bound, although for VIX, the values have some "soft" bounds.

In [3]:
with open("Data/Transformed_Data/datasets_v2.pickle", "rb") as f:
    ml_dataset = pickle.load(f)

print(ml_dataset.keys())
print(ml_dataset[list(ml_dataset.keys())[1]].keys())

for key in ml_dataset.keys():
    close_price_df = ml_dataset[key]["all"][["Date","Close"]]
    for key_2 in ml_dataset[key]:
        columns_to_work_with = [x for x in ml_dataset[key][key_2].columns if x!="Date"]
        if key_2 in ["all", "target"]:
            continue

        elif key_2 in ["BB", "EMA", "Max_High", "Min_Low", "High_Low_Mean", "High_Low_Diff"]:
            for col in columns_to_work_with:
                ml_dataset[key][key_2][col] = ((ml_dataset[key][key_2][col] - close_price_df['Close']) / close_price_df['Close']) * 100
                ml_dataset[key]["all"][col] = ((ml_dataset[key]["all"][col] - close_price_df['Close']) / close_price_df['Close']) * 100
    

ml_dataset["all"]["BB"]

dict_keys(['all', 'train', 'val', 'test'])
dict_keys(['AROON', 'BB', 'EMA', 'High_Low_Diff', 'High_Low_Mean', 'MACD', 'Max_High', 'Min_Low', 'ROC', 'RSI', 'all', 'target'])


,Date,BB_upper_3,BB_middle_3,BB_lower_3,BB_upper_6,BB_middle_6,BB_lower_6,BB_upper_9,BB_middle_9,BB_lower_9,BB_upper_12,BB_middle_12,BB_lower_12,BB_upper_18,BB_middle_18,BB_lower_18,BB_upper_24,BB_middle_24,BB_lower_24,BB_upper_30,BB_middle_30,BB_lower_30
0,2003-01-02,24.012955,9.793886,-4.425182,21.998723,9.649472,-2.699780,22.421156,9.474424,-3.472308,20.756693,8.753449,-3.249795,21.697900,10.060832,-1.576237,21.252795,10.371540,-0.509714,22.583086,7.955891,-6.671304
1,2003-01-03,20.175548,6.280388,-7.614771,27.226839,11.581577,-4.063685,23.583987,10.134161,-3.315665,25.071375,11.355347,-2.360680,25.526855,12.281649,-0.963558,25.502690,13.112843,0.722997,26.256054,10.692867,-4.870319
2,2003-01-06,2.709366,0.334538,-2.040290,26.322468,8.905394,-8.511681,22.817781,8.314378,-6.189026,24.411996,9.741738,-4.928520,22.574042,9.935770,-2.702502,24.816461,11.635221,-1.546019,25.160468,9.615951,-5.928566
3,2003-01-07,0.573498,-0.888710,-2.350917,20.639163,5.020564,-10.598036,21.913309,6.893049,-8.127212,22.832044,7.733124,-7.365796,21.039657,8.170849,-4.697959,24.009531,10.271592,-3.466347,23.714067,8.831414,-6.051240
4,2003-01-08,0.678770,-1.331770,-3.342310,11.137314,0.705050,-9.727213,19.934957,4.800450,-10.334057,17.686445,4.517558,-8.651328,18.940734,5.990771,-6.959192,21.987077,8.086889,-5.813299,21.040565,7.435694,-6.169177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5197,2023-08-25,12.253422,3.869047,-4.515328,14.679485,6.568874,-1.541738,15.537930,7.277490,-0.982949,16.634302,4.644980,-7.344343,14.510742,4.071001,-6.368741,17.069414,-0.063778,-17.196970,15.841812,-2.610547,-21.062905
5198,2023-08-28,17.845783,6.012381,-5.821020,19.066439,8.355437,-2.355565,21.768950,10.529030,-0.710890,21.648810,8.383067,-4.882676,19.318144,7.839670,-3.638805,21.438398,4.249558,-12.939282,20.300535,1.618037,-17.064460
5199,2023-08-29,11.241460,4.290659,-2.660142,23.436568,9.988467,-3.459634,28.694932,13.556324,-1.582284,27.464831,12.883506,-1.697819,25.239748,11.976164,-1.287420,25.981158,9.158017,-7.665124,25.249705,6.313726,-12.622252
5200,2023-08-30,11.312707,4.250719,-2.811269,26.322236,10.794908,-4.732419,32.448001,15.009604,-2.428794,34.071253,16.954849,-0.161555,30.625962,15.285781,-0.054400,31.414031,13.481627,-4.450777,30.356892,10.708452,-8.939988


In [3]:
ml_dataset["all"]["all"]

,Date,Open,High,Low,Close,AROON_down_3,AROON_up_3,AROON_down_6,AROON_up_6,AROON_down_9,AROON_up_9,AROON_down_12,AROON_up_12,AROON_down_18,AROON_up_18,AROON_down_24,AROON_up_24,AROON_down_30,AROON_up_30,BB_upper_3,BB_middle_3,BB_lower_3,BB_upper_6,BB_middle_6,BB_lower_6,BB_upper_9,BB_middle_9,BB_lower_9,BB_upper_12,BB_middle_12,BB_lower_12,BB_upper_18,BB_middle_18,BB_lower_18,BB_upper_24,BB_middle_24,BB_lower_24,BB_upper_30,BB_middle_30,BB_lower_30,EMA_3,EMA_6,EMA_9,EMA_12,EMA_18,EMA_24,EMA_30,EMA_36,EMA_42,EMA_48,EMA_54,EMA_60,High_Low_Diff_3,High_Low_Diff_6,High_Low_Diff_9,High_Low_Diff_12,High_Low_Diff_18,High_Low_Diff_24,High_Low_Diff_30,High_Low_Diff_36,High_Low_Diff_42,High_Low_Diff_48,High_Low_Diff_54,High_Low_Diff_60,High_Low_Mean_3,High_Low_Mean_6,High_Low_Mean_9,High_Low_Mean_12,High_Low_Mean_18,High_Low_Mean_24,High_Low_Mean_30,High_Low_Mean_36,High_Low_Mean_42,High_Low_Mean_48,High_Low_Mean_54,High_Low_Mean_60,MACD_3,MACD_signal_3,MACD_hist_3,MACD_6,MACD_signal_6,MACD_hist_6,MACD_9,MACD_signal_9,MACD_hist_9,MACD_12,MACD_signal_12,MACD_hist_12,MACD_18,MACD_signal_18,MACD_hist_18,MACD_24,MACD_signal_24,MACD_hist_24,MACD_30,MACD_signal_30,MACD_hist_30,Max_High_3,Max_High_6,Max_High_9,Max_High_12,Max_High_18,Max_High_24,Max_High_30,Max_High_36,Max_High_42,Max_High_48,Max_High_54,Max_High_60,Min_Low_3,Min_Low_6,Min_Low_9,Min_Low_12,Min_Low_18,Min_Low_24,Min_Low_30,Min_Low_36,Min_Low_42,Min_Low_48,Min_Low_54,Min_Low_60,ROC_1,ROC_2,ROC_3,ROC_6,ROC_9,ROC_12,ROC_18,ROC_24,ROC_30,RSI_3,RSI_6,RSI_9,RSI_12,RSI_18,RSI_24,RSI_30
0,2003-01-02,28.740000,28.740000,25.320000,25.389999,100.000000,33.333333,100.000000,66.666667,100.000000,11.111111,100.000000,33.333333,100.000000,11.111111,4.166667,33.333333,13.333333,46.666667,24.012955,9.793886,-4.425182,21.998723,9.649472,-2.699780,22.421156,9.474424,-3.472308,20.756693,8.753449,-3.249795,21.697900,10.060832,-1.576237,21.252795,10.371540,-0.509714,22.583086,7.955891,-6.671304,6.697600,8.600178,9.033611,9.227874,9.599846,10.268226,11.227509,12.335439,13.461130,14.497853,15.458435,16.390086,-78.298541,-78.298541,-78.259155,-78.259155,-76.841271,-71.524216,-67.900742,-65.143755,-62.386769,-55.179199,-47.932251,-19.692793,10.575031,10.575031,10.594725,10.594725,11.303666,8.645139,6.833402,8.211895,9.590388,13.194173,16.817647,30.937376,-0.328759,0.002245,-0.331004,-0.149031,-0.043062,-0.105969,-0.338705,-0.389777,0.051073,-0.684369,-0.882438,0.198069,-1.236199,-1.457601,0.221402,-1.468834,-1.417065,-0.051769,-1.380880,-0.951390,-0.429490,21.425761,21.425761,21.465147,21.465147,22.883031,22.883031,22.883031,25.640017,28.397004,35.604573,42.851522,71.090979,-0.275698,-0.275698,-0.275698,-0.275698,-0.275698,-5.592753,-9.216228,-9.216228,-9.216228,-9.216228,-9.216228,-9.216228,-11.285819,-14.280896,-14.077834,-3.091608,-10.250977,-9.900642,-15.647844,-2.233346,-8.206799,19.691252,34.252023,39.191736,41.396219,43.241865,44.185714,44.972919
1,2003-01-03,25.549999,25.549999,24.680000,24.680000,100.000000,0.000000,100.000000,50.000000,100.000000,0.000000,100.000000,25.000000,100.000000,5.555556,0.000000,29.166667,10.000000,43.333333,20.175548,6.280388,-7.614771,27.226839,11.581577,-4.063685,23.583987,10.134161,-3.315665,25.071375,11.355347,-2.360680,25.526855,12.281649,-0.963558,25.502690,13.112843,0.722997,26.256054,10.692867,-4.870319,4.883549,8.374578,9.736249,10.467061,11.410432,12.365208,13.496529,14.725660,15.947286,17.065555,18.097058,19.091257,-79.983791,-75.081039,-75.081039,-75.040520,-73.581846,-73.581846,-66.977304,-64.141004,-61.304704,-57.536469,-49.108585,-20.867089,10.008105,12.459480,12.459480,12.479740,13.209077,13.209077,9.906806,11.324956,12.743106,14.627223,18.841165,32.961913,-0.797982,-0.397868,-0.400114,-0.463417,-0.163163,-0.300254,-0.557156,-0.423253,-0.133903,-0.839444,-0.875824,0.036380,-1.329759,-1.444144,0.114385,-1.544045,-1.427223,-0.116822,-1.453974,-0.983815,-0.470159,20.016209,24.918961,24.918961,24.959480,26.418154,26.418154,26.418154,29.254454,32.090754,35.8

In [4]:
# Checkpoint

with open("Data/Transformed_Data/datasets_v3.pickle", "wb") as f:
    pickle.dump(ml_dataset, f)

with open("Data/Transformed_Data/datasets_v3.pickle", "rb") as f:
    ml_dataset = pickle.load(f)

### Checking for Nan values in features

In [5]:

for key in ml_dataset.keys():
    for key_2 in ml_dataset[key]:
        if key_2 in ["target"]:
            continue
        if ml_dataset[key][key_2].isna().sum().sum() > 0:
            print("")
            print(f"NaN values found in {key}{key_2}")
            print(ml_dataset[key][key_2].isna().sum())
            print("")
        else: 
            print(f"No NaN values found in {key} {key_2}")


No NaN values found in all AROON
No NaN values found in all BB
No NaN values found in all EMA
No NaN values found in all High_Low_Diff
No NaN values found in all High_Low_Mean
No NaN values found in all MACD
No NaN values found in all Max_High
No NaN values found in all Min_Low
No NaN values found in all ROC
No NaN values found in all RSI
No NaN values found in all all
No NaN values found in train AROON
No NaN values found in train BB
No NaN values found in train EMA
No NaN values found in train High_Low_Diff
No NaN values found in train High_Low_Mean
No NaN values found in train MACD
No NaN values found in train Max_High
No NaN values found in train Min_Low
No NaN values found in train ROC
No NaN values found in train RSI
No NaN values found in train all
No NaN values found in val AROON
No NaN values found in val BB
No NaN values found in val EMA
No NaN values found in val High_Low_Diff
No NaN values found in val High_Low_Mean
No NaN values found in val MACD
No NaN values found in val

### Normalization

In the section below i will temporally normalize the training data, such that at any given point $t$, the data used for the normalization will only look backwards, which is done to avoid information leakage. 
Consider whether this practice is a good idea, and whether it would make a difference. 

Arguments for:
- Avoid data leakage

Arguments against: 
- idk it seems to make sense to me and it's not that hard to implement

In [6]:
normalization_df = pd.DataFrame(columns=["Indicator", "Mean", "Std"])
new_ml_dataset = copy.deepcopy(ml_dataset)
indicator_col = []
mean_col = []
std_col = []


key = "train"
for key_2 in tqdm(ml_dataset[key] , total = len(ml_dataset[key].keys())):
    if key_2 in ["target"]:
        continue
    elif key_2 in ["all"]:
        columns_to_work_with = ["Open", "High", "Low", "Close"]
    else:
        columns_to_work_with = [x for x in ml_dataset[key][key_2].columns if x!="Date"]
    for col in columns_to_work_with:
        current_column = []
        list_of_values = ml_dataset[key][key_2][col].tolist()
        for index in range(len(ml_dataset["train"]["all"])):
            if index <= 1:
                current_column.append(0) 
                continue
            else:
                current_series = ml_dataset[key][key_2][col].iloc[0:index]
                specific_mean  = current_series.mean()
                specific_std   = current_series.std()
                if specific_std == 0:
                    specific_std = 1
            current_column.append((list_of_values[index] - specific_mean) / specific_std)

        new_ml_dataset[key][key_2][col] = current_column
        new_ml_dataset[key]["all"][col] = current_column
        
        indicator_col.append(col)
        mean_col.append(specific_mean)
        std_col.append(specific_std)

normalization_df["Indicator"] = indicator_col
normalization_df["Mean"] = mean_col
normalization_df["Std"] = std_col

normalization_df.to_csv("Data/Transformed_Data/normalization_df.csv", index=False)
normalization_df
                

  0%|          | 0/12 [00:00<?, ?it/s]

,Indicator,Mean,Std
0,AROON_down_3,52.512141,40.590896
1,AROON_up_3,48.167770,40.728601
2,AROON_down_6,53.024283,37.195348
3,AROON_up_6,47.818985,37.340166
4,AROON_down_9,53.548197,35.889355
5,AROON_up_9,47.228845,35.811863
6,AROON_down_12,53.869757,35.068698
7,AROON_up_12,47.130243,35.166514
8,AROON_down_18,54.347314,34.292617
9,AROON_up_18,47.477557,34.477813


In [7]:
for key in ["val","test","all"]: # Hardcoded on purpose as we don't want to use the train key in this loop and we want to use this specific order.
    for key_2 in tqdm(ml_dataset[key] , total = len(ml_dataset[key].keys())):
        if key_2 in ["target"]:
            continue
        elif key_2 in ["all"]:
            columns_to_work_with = ["Open", "High", "Low", "Close"]
        else:
            columns_to_work_with = [x for x in ml_dataset[key][key_2].columns if x!="Date"]
        train_dates = (ml_dataset["train"][key_2]["Date"].iloc[0], ml_dataset["train"][key_2]["Date"].iloc[-1])
        val_dates   = (ml_dataset["val"][key_2]["Date"].iloc[0], ml_dataset["val"][key_2]["Date"].iloc[-1])
        test_dates  = (ml_dataset["test"][key_2]["Date"].iloc[0], ml_dataset["test"][key_2]["Date"].iloc[-1])

        for col in columns_to_work_with:
            specific_mean = normalization_df[normalization_df["Indicator"] == col]["Mean"].reset_index(drop=True)[0]
            specific_std  = normalization_df[normalization_df["Indicator"] == col]["Std"].reset_index(drop=True)[0]

            # For test, the specific_mean and specific_std to normalize the data
            if key in ["val","test"]:
                new_ml_dataset[key][key_2][col] = (ml_dataset[key][key_2][col] - specific_mean) / specific_std
                new_ml_dataset[key]["all"][col] = (ml_dataset[key]["all"][col] - specific_mean) / specific_std
            elif key == "all":
                # overall task: Copy the data from train, val and test to all, where the dates match.
                train_col = new_ml_dataset["train"][key_2][col]
                val_col = new_ml_dataset["val"][key_2][col]
                test_col = new_ml_dataset["test"][key_2][col]

                all_col = pd.concat([train_col, val_col, test_col], axis=0).reset_index(drop=True)
                new_ml_dataset[key][key_2][col] = all_col
                new_ml_dataset[key]["all"][col] = all_col

            else: 
                raise KeyError("Something went wrong as all of the values for the (key) variable are supposed to be handled by the conditions above")


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [8]:
with open("Data/Transformed_Data/datasets_v4.pickle", "wb") as f:
    pickle.dump(new_ml_dataset, f)

with open("Data/Transformed_Data/datasets_v4.pickle", "rb") as f:
    new_ml_dataset = pickle.load(f)

In [9]:
new_ml_dataset["train"]["all"] 

,Date,Open,High,Low,Close,AROON_down_3,AROON_up_3,AROON_down_6,AROON_up_6,AROON_down_9,AROON_up_9,AROON_down_12,AROON_up_12,AROON_down_18,AROON_up_18,AROON_down_24,AROON_up_24,AROON_down_30,AROON_up_30,BB_upper_3,BB_middle_3,BB_lower_3,BB_upper_6,BB_middle_6,BB_lower_6,BB_upper_9,BB_middle_9,BB_lower_9,BB_upper_12,BB_middle_12,BB_lower_12,BB_upper_18,BB_middle_18,BB_lower_18,BB_upper_24,BB_middle_24,BB_lower_24,BB_upper_30,BB_middle_30,BB_lower_30,EMA_3,EMA_6,EMA_9,EMA_12,EMA_18,EMA_24,EMA_30,EMA_36,EMA_42,EMA_48,EMA_54,EMA_60,High_Low_Diff_3,High_Low_Diff_6,High_Low_Diff_9,High_Low_Diff_12,High_Low_Diff_18,High_Low_Diff_24,High_Low_Diff_30,High_Low_Diff_36,High_Low_Diff_42,High_Low_Diff_48,High_Low_Diff_54,High_Low_Diff_60,High_Low_Mean_3,High_Low_Mean_6,High_Low_Mean_9,High_Low_Mean_12,High_Low_Mean_18,High_Low_Mean_24,High_Low_Mean_30,High_Low_Mean_36,High_Low_Mean_42,High_Low_Mean_48,High_Low_Mean_54,High_Low_Mean_60,MACD_3,MACD_signal_3,MACD_hist_3,MACD_6,MACD_signal_6,MACD_hist_6,MACD_9,MACD_signal_9,MACD_hist_9,MACD_12,MACD_signal_12,MACD_hist_12,MACD_18,MACD_signal_18,MACD_hist_18,MACD_24,MACD_signal_24,MACD_hist_24,MACD_30,MACD_signal_30,MACD_hist_30,Max_High_3,Max_High_6,Max_High_9,Max_High_12,Max_High_18,Max_High_24,Max_High_30,Max_High_36,Max_High_42,Max_High_48,Max_High_54,Max_High_60,Min_Low_3,Min_Low_6,Min_Low_9,Min_Low_12,Min_Low_18,Min_Low_24,Min_Low_30,Min_Low_36,Min_Low_42,Min_Low_48,Min_Low_54,Min_Low_60,ROC_1,ROC_2,ROC_3,ROC_6,ROC_9,ROC_12,ROC_18,ROC_24,ROC_30,RSI_3,RSI_6,RSI_9,RSI_12,RSI_18,RSI_24,RSI_30
0,2003-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2003-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [10]:
new_ml_dataset["all"]["BB"]

,Date,BB_upper_3,BB_middle_3,BB_lower_3,BB_upper_6,BB_middle_6,BB_lower_6,BB_upper_9,BB_middle_9,BB_lower_9,BB_upper_12,BB_middle_12,BB_lower_12,BB_upper_18,BB_middle_18,BB_lower_18,BB_upper_24,BB_middle_24,BB_lower_24,BB_upper_30,BB_middle_30,BB_lower_30
0,2003-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2003-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2003-01-06,-7.143982,-3.100364,1.764530,0.462473,-1.251738,-5.319170,-0.224738,-3.193784,-25.234322,0.490983,-0.169941,-3.377265,-0.383506,-0.786746,-3.306804,0.478754,-0.055185,-1.895996,0.285270,0.150657,-0.123875
3,2003-01-07,-1.326194,-1.329771,0.837532,-1.626042,-3.637706,-1.811723,-1.738497,-2.620917,-2.353000,-0.250108,-1.688009,-2.954607,-1.109896,-1.961373,-3.345185,0.066716,-1.045845,-2.660480,-0.505116,-0.428021,-0.251737
4,2003-01-08,-0.936812,-1.035160,0.298258,-4.009626,-2.935439,-0.879248,-3.899630,-2.744207,-2.186137,-2.907283,-3.162921,-1.896882,-1.902961,-2.447226,-2.725369,-1.023534,-2.451213,-2.602119,-2.102177,-1.579576,-0.386263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5197,2023-08-25,0.676696,0.750260,0.320295,0.335973,0.822758,0.919454,0.151042,0.739020,1.047543,0.079638,0.375236,0.539309,-0.248784,0.240801,0.729769,-0.214328,-0.140709,0.018900,-0.371073,-0.342431,-0.147293
5198,2023-08-28,1.437139,1.193955,0.136226,0.752747,1.066535,0.831659,0.648568,1.108909,1.073215,0.441060,0.758429,0.752887,0.051287,0.574866,0.941286,0.034325,0.205970,0.323133,-0.136111,-0.028743,0.121573
5199,2023-08-29,0.539090,0.837538,0.581829,1.167923,1.289362,0.712554,1.201585,1.453288,0.990988,0.860251,1.219770,1.029210,0.420906,0.941537,1.123471,0.292868,0.600480,0.699997,0.124696,0.319597,0.420279
5200,2023-08-30,0.548778,0.829270,0.560524,1.442070,1.399401,0.575247,1.501256,1.618611,0.911108,1.336410,1.637126,1.162498,0.757106,1.234912,1.219006,0.602070,0.947984,0.929678,0.393831,0.645610,0.667885
